In [1]:
!pip install selenium beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 9.9 MB/s eta 0:00:00


In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import re
import time


# 株価データを抽出する関数
def extract_stock_data(nikkeihtml):
    soup = BeautifulSoup(nikkeihtml)

    stock_lists = [0,0,0,0,0]
    #情報取得
    for i in range(1,6):
        st_data = soup.select_one(f'#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child({i})')
        st_data_value = st_data.text

        #文字列をdatetimeに変換
        if  i == 1:
            date = datetime.strptime(st_data_value,'%Y/%m/%d')
            st_data_value = date.strftime('%Y-%m-%d')

        stock_lists[i-1] = st_data_value

    return stock_lists


# 株価データを取得する関数。
def get_stock_values(chrome_driver):

    chrome_driver.get('https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month')

    # グラフ要素を最大で10回の試行で取得する
    for _ in range(10):
        graph = chrome_driver.find_elements(By.CSS_SELECTOR, '.highcharts-grid')[1]
        if graph is not None:
            break
        print('continue find elements')

    # マウスの動きを記述
    actions = ActionChains(chrome_driver)

    # グラフの中心にマウスを移動
    actions.move_to_element(graph)

    # グラフの横幅（width）の半分の値だけ右にマウスをずらし、グラフの右端にマウスを移動
    actions.move_by_offset((graph.rect['width'] // 2),0)

    # 記述した動作の実行
    actions.perform()

    # グラフの横幅を取得
    graph_width     = graph.rect['width']

    stock_values    = []
    html            = chrome_driver.page_source.encode('utf-8')

    # 株価データを抽出する。
    stock_values.append(extract_stock_data(html))

    # num_points = 30  # 取りたいデータポイントの数を仮定
    # step = graph_width // num_points

    # グラフを1ピクセル左に移動しながら、株価データを抽出する
    #for k in range(num_points):
    for k in range(graph_width - 1):
        actions         = ActionChains(chrome_driver)

        #actions.move_by_offset(-step, 0).perform()
        actions.move_by_offset(-1, 0).perform()

        html            = chrome_driver.page_source.encode('utf-8')
        temp_value      = extract_stock_data(html)
        #print(temp_value)

        if temp_value not in stock_values:
            stock_values.append(temp_value)

    return stock_values


# ヘッドレスモードで起動するためのオプションを設定
chrome_options  = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Chromeを立ち上げる
chrome_driver   = webdriver.Chrome(options=chrome_options)

start_time = time.time()
print(f'Scraping time: {time.time() - start_time}')
result = get_stock_values(chrome_driver)

# ブラウザを閉じる
chrome_driver.quit()

# 結果を表示する
print('日付, 始値, 高値, 安値, 終値')
for data in result:
    print(', '.join(data))

Scraping time: 9.560585021972656e-05
日付, 始値, 高値, 安値, 終値
2025-01-22, 始値: 39,355.22, 高値: 39,647.83, 安値: 39,332.63, 終値: 39,629.38
2025-01-21, 始値: 39,163.53, 高値: 39,238.21, 安値: 38,643.84, 終値: 39,027.98
2025-01-20, 始値: 38,671.77, 高値: 39,032.93, 安値: 38,671.77, 終値: 38,902.5
2025-01-17, 始値: 38,454.1, 高値: 38,503.94, 安値: 38,055.68, 終値: 38,451.46
2025-01-16, 始値: 38,732.65, 高値: 38,932.54, 安値: 38,426.2, 終値: 38,572.6
2025-01-15, 始値: 38,721.61, 高値: 38,774.99, 安値: 38,316.01, 終値: 38,444.58
2025-01-14, 始値: 39,010.96, 高値: 39,054.35, 安値: 38,305.91, 終値: 38,474.3
2025-01-10, 始値: 39,550.25, 高値: 39,591.46, 安値: 39,166.05, 終値: 39,190.4
2025-01-09, 始値: 39,888.91, 高値: 39,930.07, 安値: 39,385.05, 終値: 39,605.09
2025-01-08, 始値: 39,879.36, 高値: 40,105.72, 安値: 39,705.81, 終値: 39,981.06
2025-01-07, 始値: 39,584.36, 高値: 40,288.8, 安値: 39,584.36, 終値: 40,083.3
2025-01-06, 始値: 39,945.42, 高値: 39,993.5, 安値: 39,232.78, 終値: 39,307.05
2024-12-30, 始値: 40,325.78, 高値: 40,325.78, 安値: 39,864.46, 終値: 39,894.54
2024-12-27, 始値: 39,672.15, 高値: